In [6]:
!head Ecommerce_FAQ_Chatbot_dataset.json

{
  "questions": [
  {
  "question": "How can I create an account?",
  "answer": "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."
  },
  {
  "question": "What payment methods do you accept?",
  "answer": "We accept major credit cards, debit cards, and PayPal as payment methods for online orders."
  },


## Generating unique ID's

In [2]:
# Re-load the document since it was not defined in this new code execution.
import json
import hashlib

with open('Ecommerce_FAQ_Chatbot_dataset.json', 'rt') as f_in:
    documents= json.load(f_in)

def generate_document_id(doc):
    combined = f"{doc['question']}-{doc['answer'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

for qa_pair in documents['questions']:
    doc_id = generate_document_id(qa_pair)
    qa_pair['id'] = doc_id  # Add the generated ID to the question-answer pair


In [6]:
hashes = []

for doc in documents['questions']:
    hashes.append(doc)

In [27]:
hashes[0]

{'question': 'How can I create an account?',
 'answer': "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.",
 'id': 'a5825c73'}

In [24]:
# check if hash unique hash id count and len of questions ar equal
len(hash_list) == len(documents['questions'])

True

In [30]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(hashes, f_out, indent=2)

In [31]:
!head documents-with-ids.json

[
  {
    "question": "How can I create an account?",
    "answer": "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.",
    "id": "a5825c73"
  },
  {
    "question": "What payment methods do you accept?",
    "answer": "We accept major credit cards, debit cards, and PayPal as payment methods for online orders.",
    "id": "db025979"


## Chunking

In [17]:
from openai import OpenAI
from dotenv import load_dotenv

In [18]:
load_dotenv()

True

In [19]:
import json
with open("documents-with-ids.json",'rt') as f_out:
    documents= json.load(f_out)

In [20]:
documents[0]

{'question': 'How can I create an account?',
 'answer': "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.",
 'id': 'a5825c73'}

In [44]:
prompt=f"""Review {documents} and create output a new .json file that contains a key which categorises the type of question
            based on the topics they belong to, some high level topics are:
                1. Account & Registration
                2. Payments & Pricing
                3. Shipping & Delivery
                4. Returns & Refunds
                5. Order Management
                6. Customer Support & Services
                7. Product Information
                
            The topics above must be introduced as a new key "topic" for each question, answer pair.
            
            return only the contents found in between ```json``` nothing else.
"""

In [202]:

client= OpenAI()
model= "gpt-4o-mini"
response= client.chat.completions.create(
        model=model,
       messages=[
        # {"role": "system", "content": "You are a helpful assistant."},
        {"role":"user", "content": prompt}
    ],
)

NameError: name 'OpenAI' is not defined

In [95]:
def format_output(response):
    data_to_view = {
        "response": response.choices[0].message.content.replace("```json",'').replace('```','')
    }
    return data_to_view['response']

response_string = format_output(response)

In [134]:
json.loads(response_string)

[{'question': 'How can I create an account?',
  'answer': "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.",
  'id': 'a5825c73',
  'topic': 'Account & Registration'},
 {'question': 'What payment methods do you accept?',
  'answer': 'We accept major credit cards, debit cards, and PayPal as payment methods for online orders.',
  'id': 'db025979',
  'topic': 'Payments & Pricing'},
 {'question': 'How can I track my order?',
  'answer': "You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment.",
  'id': 'f936d8f6',
  'topic': 'Order Management'},
 {'question': 'What is your return policy?',
  'answer': 'Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our R

## Indexing

In [201]:
import minsearch